In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from psql_helpers import get_schema_info,init_db, set_default_access,get_accessible_meetings,get_session

In [3]:
from psql_models import AccessLevel

In [17]:
olga_token = '5faa86ba0dc74b14a78c60c8aa20a014'
olga_id = '8b767b85-44fb-4c20-97df-7aa172242273'
from vexa import VexaAPI





In [14]:
user_info

{'id': 'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
 'email': '2280905@gmail.com',
 'is_allowed_send_init_message': True,
 'username': 'Dmitriy Grankin',
 'first_name': 'Dmitriy',
 'last_name': 'Grankin',
 'image': 'https://lh3.googleusercontent.com/a/ACg8ocLu4ZPfzj9KFNVqKL3NTF4SXUYvJ4mwoXLuoQ0BZzjw7OBIRA=s96-c',
 'created_timestamp': '2024-05-28T11:16:50.894711Z',
 'updated_timestamp': '2024-10-18T08:47:02.519781Z'}

In [5]:
await init_db()

Database initialized successfully.


In [6]:
await get_schema_info()

[('alembic_version', ['version_num character varying NOT NULL', 'version_num character varying NOT NULL'], ['PK: version_num', 'CHECK: 2200_25737_1_not_null']),
 ('default_access', ['owner_user_id uuid NOT NULL', 'owner_user_id uuid NOT NULL', 'owner_user_id uuid NOT NULL', 'owner_user_id uuid NOT NULL', 'owner_user_id uuid NOT N ... (1520 characters truncated) ... id uuid NOT NULL', 'granted_user_id uuid NOT NULL', 'granted_user_id uuid NOT NULL', 'granted_user_id uuid NOT NULL', 'granted_user_id uuid NOT NULL'], ['FK: owner_user_id -> users.id', 'FK: granted_user_id -> users.id', 'PK: owner_user_id', 'PK: owner_user_id', 'PK: granted_user_id', 'PK: granted_use ... (856 characters truncated) ... granted_user_id', 'CHECK: valid_default_access_level', 'CHECK: 2200_26957_2_not_null', 'CHECK: 2200_26957_3_not_null', 'CHECK: 2200_26957_1_not_null']),
 ('discussion_points', ['speaker_id integer', 'speaker_id integer', 'speaker_id integer', 'speaker_id integer', 'meeting_id uuid', 'meeting_i

In [11]:
from psql_helpers import create_user

In [18]:
vexa = VexaAPI(olga_token)
user_info = await vexa.get_user_info()

async with get_session() as session:
    # Filter and rename fields to match create_user parameters
    filtered_user_info = {
        'user_id': user_info['id'],
        'email': user_info['email'],
        'username': user_info['username'],
        'first_name': user_info['first_name'],
        'last_name': user_info['last_name'],
        'image': user_info['image']
    }
    await create_user(session, **filtered_user_info)

User information retrieved successfully.


In [29]:
from psql_helpers import read_table_async

In [30]:
from psql_models import User,DefaultAccess

In [31]:
await read_table_async(User)

,id,email,username,first_name,last_name,image,created_timestamp,updated_timestamp
0,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,2280905@gmail.com,Dmitriy Grankin,Dmitriy,Grankin,https://lh3.googleusercontent.com/a/ACg8ocLu4Z...,2024-10-29 14:02:16.627179+00:00,2024-10-29 14:02:16.627181+00:00
1,8b767b85-44fb-4c20-97df-7aa172242273,olganemirovskaya@gmail.com,Olga Nemirovskaya,Olga,Nemirovskaya,https://lh3.googleusercontent.com/a/ACg8ocKR1i...,2024-10-29 14:02:58.711841+00:00,2024-10-29 14:02:58.711844+00:00
2,fc97506f-0e97-409d-8bdd-699d6dd330a4,user1@example.com,user1,First1,Last1,None,2024-10-29 14:16:41.074831+00:00,2024-10-29 14:16:41.074833+00:00
3,b15fd48d-e746-41a1-934b-f70cab046c2e,user2@example.com,user2,First2,Last2,None,2024-10-29 14:16:41.074836+00:00,2024-10-29 14:16:41.074836+00:00
4,5eaa1e88-0d20-4986-9129-919d1227db52,user3@example.com,user3,First3,Last3,None,2024-10-29 14:16:41.074845+00:00,2024-10-29 14:16:41.074846+00:00


In [32]:
await read_table_async(DiscussionPoint)

NameError: name 'DiscussionPoint' is not defined

In [26]:
await set_default_access(
        session,
        owner_user_id='ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
        granted_user_id='8b767b85-44fb-4c20-97df-7aa172242273',
        access_level=AccessLevel.SEARCH
    )

In [28]:
await read_table_async(DefaultAccess)

,owner_user_id,granted_user_id,access_level,created_at
0,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,8b767b85-44fb-4c20-97df-7aa172242273,search,2024-10-29 14:04:30.543456+00:00


In [ ]:
async with get_session() as session:
    # First create the users if they don't exist
    owner = await create_user(session, owner_user_id, "Owner Name")
    granted = await create_user(session, granted_user_id, "Granted User Name")
    
    # Then set the default access
    await set_default_access(
        session,
        owner_user_id=owner_user_id,
        granted_user_id=granted_user_id,
        access_level=AccessLevel.SEARCH
    )

In [ ]:
async with get_session() as session:
    # First create the users if they don't exist
    owner = await create_user(session, owner_user_id, "Owner Name")
    granted = await create_user(session, granted_user_id, "Granted User Name")
    
    # Then set the default access
    await set_default_access(
        session,
        owner_user_id=owner_user_id,
        granted_user_id=granted_user_id,
        access_level=AccessLevel.SEARCH
    )

In [18]:
async with get_session() as session:
    await set_default_access(
        session,
        owner_user_id=vexa.user_id,
        granted_user_id=olga_id,
        access_level=AccessLevel.SEARCH
    )


IntegrityError: (sqlalchemy.dialects.postgresql.asyncpg.IntegrityError) <class 'asyncpg.exceptions.ForeignKeyViolationError'>: insert or update on table "default_access" violates foreign key constraint "default_access_owner_user_id_fkey"
DETAIL:  Key (owner_user_id)=(ef7c085b-fdb5-4c94-b7b6-a61a3d04c210) is not present in table "users".
[SQL: INSERT INTO default_access (owner_user_id, granted_user_id, access_level, created_at) VALUES ($1::UUID, $2::UUID, $3::VARCHAR, $4::TIMESTAMP WITH TIME ZONE)]
[parameters: ('ef7c085b-fdb5-4c94-b7b6-a61a3d04c210', '8b767b85-44fb-4c20-97df-7aa172242273', 'search', datetime.datetime(2024, 10, 29, 16, 49, 57, 38839))]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [9]:
from psql_models import get_accessible_meetings,get_session,set_default_access,AccessLevel,read_table_async,DefaultAccess,UserMeeting

In [10]:
await read_table_async(DefaultAccess)

""


In [11]:
await read_table_async(UserMeeting)

""


In [5]:
async with get_session() as session:
    # Set default search access
    await set_default_access(
        session,
        owner_user_id=vexa.user_id,
        granted_user_id='ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
        access_level=AccessLevel.TRANSCRIPT
    )

In [ ]:
'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210'

In [21]:
vexa.user_id

'8b767b85-44fb-4c20-97df-7aa172242273'

In [7]:
async with get_session() as session:
    meetings, total = await get_accessible_meetings(session, vexa.user_id)
    print(meetings)
    print(total)

[<psql_models.Meeting object at 0x7f4224f1fc50>, <psql_models.Meeting object at 0x7f4224f10c90>, <psql_models.Meeting object at 0x7f4224f1fd10>, <psql_models.Meeting object at 0x7f4224f1fd90>, <psql_models.Meeting object at 0x7f4224f1fe10>, <psql_models.Meeting object at 0x7f4224f1fed0>, <psql_models.Meeting object at 0x7f4224f1ff50>, <psql_models.Meeting object at 0x7f4224f1ffd0>, <psql_models.Meeting object at 0x7f4224f34090>, <psql_models.Meeting object at 0x7f4224f1fe90>, <psql_models.Meeting object at 0x7f4224f34150>, <psql_models.Meeting object at 0x7f4224f341d0>, <psql_models.Meeting object at 0x7f4224f34250>, <psql_models.Meeting object at 0x7f4224f342d0>, <psql_models.Meeting object at 0x7f4224f34350>, <psql_models.Meeting object at 0x7f4224f343d0>, <psql_models.Meeting object at 0x7f4224f34450>, <psql_models.Meeting object at 0x7f4224f344d0>, <psql_models.Meeting object at 0x7f4224f34590>, <psql_models.Meeting object at 0x7f4224f34690>, <psql_models.Meeting object at 0x7f4224

In [9]:
meetings[0].

DetachedInstanceError: Parent instance <Meeting at 0x7f4224f1fc50> is not bound to a Session; lazy load operation of attribute 'owner' cannot proceed (Background on this error at: https://sqlalche.me/e/20/bhk3)

In [1]:
from psql_models import get_schema_info

In [6]:
await get_schema_info()

[('alembic_version', ['version_num character varying NOT NULL', 'version_num character varying NOT NULL'], ['PK: version_num', 'CHECK: 2200_25737_1_not_null']),
 ('default_access', ['created_at timestamp with time zone', 'created_at timestamp with time zone', 'created_at timestamp with time zone', 'created_at timestamp with time  ... (1520 characters truncated) ... er_user_id uuid NOT NULL', 'owner_user_id uuid NOT NULL', 'owner_user_id uuid NOT NULL', 'owner_user_id uuid NOT NULL', 'owner_user_id uuid NOT NULL'], ['FK: granted_user_id -> users.id', 'FK: owner_user_id -> users.id', 'CHECK: 2200_26513_3_not_null', 'PK: owner_user_id', 'PK: owner_user_id', 'PK: gr ... (856 characters truncated) ... d', 'PK: granted_user_id', 'PK: granted_user_id', 'CHECK: valid_default_access_level', 'CHECK: 2200_26513_1_not_null', 'CHECK: 2200_26513_2_not_null']),
 ('discussion_points', ['summary_index integer', 'summary_index integer', 'summary_index integer', 'summary_index integer', 'summary text', '

In [1]:
import pandas as pd
from typing import Optional, List
from core import system_msg, user_msg, assistant_msg, generic_call_stream
from qdrant_search import QdrantSearchEngine

from psql_models import get_session

In [2]:
engine = QdrantSearchEngine()

In [ ]:
#await engine.initialize()
await engine.sync_from_postgres(get_session)